In [6]:
import pandas as pd
import numpy as np

train_embeddings = pd.read_csv("../artifacts/embeddings/train_embeddings.csv")
val_embeddings = pd.read_csv("../artifacts/embeddings/val_embeddings.csv")
test_embeddings = pd.read_csv("../artifacts/embeddings/test_embeddings.csv")


In [7]:
train_embeddings.head()

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_375,embedding_376,embedding_377,embedding_378,embedding_379,embedding_380,embedding_381,embedding_382,embedding_383,score
0,0.007381,-0.008330,0.016669,-0.074323,0.046939,-0.002360,-0.057948,-0.007624,-0.010701,0.009736,...,0.022288,-0.049317,-0.101603,-0.074203,-0.003890,0.027115,-0.037989,-0.037586,0.022934,0
1,-0.043017,0.017792,0.045885,-0.083600,0.097620,-0.062408,0.049754,0.059850,0.003027,0.076649,...,0.007365,0.076277,-0.103574,0.039644,0.066346,0.084934,0.016404,-0.077912,0.068454,1
2,-0.006541,-0.102944,-0.044033,-0.020572,0.012236,0.037577,0.056026,-0.017070,0.064555,-0.043743,...,-0.036510,0.020387,0.013984,0.055085,0.032886,-0.099242,-0.020611,-0.108412,-0.018762,0
3,-0.070087,-0.045394,0.000340,-0.053398,0.028893,-0.005580,-0.018931,-0.027596,-0.015632,-0.075729,...,-0.004730,0.052342,-0.045289,0.072737,0.035943,0.088658,0.033489,0.030024,0.050442,1
4,0.043325,0.045692,0.022030,-0.085556,-0.008906,0.004737,0.061751,0.029646,0.064426,-0.011236,...,0.017643,0.035781,0.040821,-0.060456,-0.010819,0.031674,-0.029560,-0.001621,-0.048673,1


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

X_train = train_embeddings.drop(columns=['score'])
y_train = train_embeddings['score']

svm = LinearSVC(C=1.0, max_iter=2000)
svm.fit(X_train, y_train)
# Make predictions
X_test = test_embeddings.drop(columns=['score'])
y_test = test_embeddings['score']
test_predictions_binary = svm.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, test_predictions_binary)
print(f"Test Accuracy: {accuracy:.4f}")

# Classification report with binary labels
print("\nClassification Report (Binary):")
print(classification_report(y_test, test_predictions_binary))


Test Accuracy: 0.8728

Classification Report (Binary):
              precision    recall  f1-score   support

           0       0.86      0.91      0.88      6979
           1       0.89      0.83      0.86      6052

    accuracy                           0.87     13031
   macro avg       0.88      0.87      0.87     13031
weighted avg       0.87      0.87      0.87     13031



In [ ]:
import optuna
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

def objective(trial):
    C = trial.suggest_float('C', 1e-3, 1e3, log=True)
    loss = trial.suggest_categorical('loss', ['hinge', 'squared_hinge'])
    dual = trial.suggest_categorical('dual', [True, False])
    class_weight = trial.suggest_categorical('class_weight', [None, 'balanced'])

    if loss == 'hinge' and dual is False:
        raise optuna.TrialPruned()

    # Define and fit model directly
    model = LinearSVC(
        C=C,
        loss=loss,
        dual=dual,
        class_weight=class_weight,
        max_iter=3000
    )

    model.fit(X_train, y_train)
    return model.score(X_val, y_val)  # Single validation split


# Run Optuna search
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

print("Best hyperparameters:", study.best_params)
print("Best validation accuracy:", study.best_value)

# Train final model with best params
best_model = LinearSVC(**study.best_params, max_iter=3000)
best_model.fit(X_train, y_train)

# Evaluate on test data
y_pred = best_model.predict(X_test)
print("\nTest Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


[I 2025-08-13 21:26:00,081] A new study created in memory with name: no-name-194f302e-52ae-4e6d-88ac-ef9076850f53
[W 2025-08-13 21:26:06,367] Trial 0 failed with parameters: {'C': 0.2026092338860802, 'loss': 'squared_hinge', 'dual': False, 'class_weight': 'balanced'} because of the following error: NameError("name 'X_val' is not defined").
Traceback (most recent call last):
  File "c:\Users\Lenovo\anaconda3\envs\myenv\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16688\3984287471.py", line 25, in objective
    return model.score(X_val, y_val)  # Single validation split
                       ^^^^^
NameError: name 'X_val' is not defined. Did you mean: 'eval'?
[W 2025-08-13 21:26:06,376] Trial 0 failed with value None.


NameError: name 'X_val' is not defined